In [ ]:
import geopandas as gpd
import pandas as pd
from keplergl import KeplerGl
from shapely.geometry import Polygon
import pickle

## Load a geo-json file into a geo-dataframe

In [ ]:
gpdf_lines = gpd.read_file('./data/madrid_lines_sample.json')
# gpdf_lines

In [ ]:
gpdf_lines.geometry.plot()

In [ ]:
gpdf_lines.explore(column='count_by_area',
                cmap="Set1")

## Create a function that turns lines into polygons with width set by number of lanes

In [ ]:
def lines_to_polygons(gpdf, epsg='32630') -> gpd.GeoDataFrame:
    LANE_WIDTH = 3.0

    # convert crs to UTM 30T (30N) which is EPSG:32630
    ways_utm_gpdf = gpdf.to_crs(epsg=epsg)
    ways_utm_gpdf.loc[:, ['lanes']] = ways_utm_gpdf.lanes.astype(int)

    # convert all lineStrings to polygons where the width is determined by 

    f = lambda x, y: x.buffer(LANE_WIDTH * float(y), cap_style=2)
    ways_utm_gpdf.geometry = ways_utm_gpdf.apply(lambda x: f(x.geometry, x.lanes), axis=1)

    # Add an "area" column with m^2 of every polygon
    ways_utm_gpdf['area_m2'] = ways_utm_gpdf.geometry.area

    # CRS now converted back to WGS84

    ways_gpdf_polygons = ways_utm_gpdf.to_crs(crs='wgs84')

    return ways_gpdf_polygons

### How it's done:

* Change CRS to UTM (enable us to use meters for width and measure area of polygons in m^2)
* Morph linestrings into polygons with width which is \<lanes\> * \<estimated width of a lane in meters\>
* Add new column that indicates the area of every polygon in m^2
* Change CRS into WGS84, back from meters to degrees. 

In [ ]:
gpdf_polygons = lines_to_polygons(gpdf_lines)


In [ ]:
gpdf_polygons.explore(column='lanes')

In [ ]:
map = KeplerGl(height=700)
map.add_data(gpdf_polygons.copy(), name='polygons')
map.add_data(gpdf_lines.copy(), name='lines')
with open('./data/map_config.bin', 'rb') as f:
    map.config = pickle.load(f)

In [ ]:
map

In [ ]:
# # re-save config
# with open('map_config.bin','+wb') as f:
#     pickle.dump(map.config, f)